# Compute gerrymandering metrics

This notebook demonstrates how to load in election results, creating 3 different DataFrames that you might be interested in:
1. Election results
2. Metrics
3. Percentiles for metrics

First, we will load in our data, both for Congressional elections and state legislative (lower house) elections.

Then we will compute the tests and generate percentile rankings, using a few parameters that we set below.

In [ ]:
import gerrymetrics as g
import IPython.display as ipd

from collections import defaultdict

# impute uncontested races at a voteshare of 0 or 1; in other words, don't impute them
impute_val = 1

# only consider races after 1972
min_year = 1972

# when identifying the worst gerrymanders:
# only examine races where D voteshare is between .45 and .55
competitiveness_threshold = .55 

# only examine races in states with at least 7 districts
min_districts = 7



chambers = defaultdict(lambda: defaultdict(list))
chambers['State Legislative']['filepath'] = 'https://raw.githubusercontent.com/PrincetonUniversity/historic_state_legislative_election_results/2bf28f2ac1a74636b09dfb700eef08a4324d2650/state_legislative_election_results_post1971.csv'
chambers['Congressional']['filepath'] = 'election_data/congressional_election_results_post1948.csv'

metric_dict = {'t_test_diff':            g.t_test_diff,
#                'mean_median_diff':       g.mean_median,
               'declination':            g.declination,
               'declination_buffered':   g.bdec,
               'efficiency_gap':         g.EG,
               'loss_gap':               g.EG_loss_only,
               'difference_gap':         g.EG_difference,
               'surplus_gap':            g.EG_surplus_only,
               'vote_centric_gap':       g.EG_vote_centric,
               'vote_centric_gap_two':   g.EG_vote_centric_two,
               'partisan_bias':          g.partisan_bias,
               'equal_vote_weight_bias': g.equal_vote_weight}

for chamber in chambers:
    print(chamber)
    chambers[chamber]['elections_df'] = g.parse_results(chambers[chamber]['filepath'])
    chambers[chamber]['tests_df'] = g.tests_df(g.run_all_tests(
        chambers[chamber]['elections_df'],
        impute_val=impute_val,
        metrics=metric_dict))
    chambers[chamber]['percentile_df'] = g.generate_percentiles(chambers[chamber]['tests_df'],
        metric_dict.keys(),
        competitiveness_threshold=competitiveness_threshold,
        min_districts=min_districts,
        min_year=min_year)

State Legislative


100%|██████████| 48/48 [00:01<00:00, 24.23it/s]


Congressional


 89%|████████▉ | 32/36 [00:02<00:00, 14.28it/s]

## 1. Election results
The first dataframe is just election results. Let's look at a few 2016 Congressional elections:

In [ ]:
chambers['Congressional']['elections_df'].loc[2016].head()

## 2. Metrics
Then we have the output from the metrics that we specified in the first cell. Let's look at those same elections again:

In [ ]:
chambers['Congressional']['tests_df'].loc[2016].head()

## 3. Percentiles for metrics
Then we have the percentile rankings for all of those metrics, using the parameters specified in the first cell.

In [ ]:
chambers['Congressional']['percentile_df'].loc[2016].head()

All of the above DataFrames are MultiIndex, so you can specify any state and year like so:

In [ ]:
chambers['Congressional']['percentile_df'].loc[2012, 'VA']

## Which elections in the ongoing cycle seem to be most gerrymandered according to these metrics?

Finally, we might be interested in which elections seem particularly gerrymandered in the ongoing 2012-2021 cycle.

In [ ]:
min_percentile = 95
min_n_tests = 3
cycle_start_year = 2012

print(
'''
Shown below are elections since {start} that rank, for
at least {min_tests} of {total_tests} metrics, in at least the {pctile}th
percentile of all elections since {min_year_overall}.

Only showing elections for states that have at least {n_districts} districts,
and for which the statewide total voteshare was competitive,
i.e., between {comp_lo:.2} and {comp_hi:.2}.
'''.format(start=cycle_start_year,
    min_tests=min_n_tests,
    total_tests=len(metric_dict),
    pctile=min_percentile,
    min_year_overall=min_year,
    n_districts=min_districts,
    comp_lo=1-competitiveness_threshold,
    comp_hi=competitiveness_threshold))

In [ ]:
for chamber in chambers:
    print('\n' + chamber + ' elections:')
    df = chambers[chamber]['percentile_df']
    cut = df[(df.loc[:, metric_dict.keys()] > min_percentile).sum(axis=1) >= min_n_tests]
    ipd.display(cut.loc[cycle_start_year:])